In [8]:
# TensorFlow 测试样例
import tensorflow as tf

a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")

result = a + b

# 并不能生产一个结果
# Tensor("add_4:0", shape=(2,), dtype=float32)
print(result)

# 需要先生成一个会话（ session),
sess = tf.Session()
print(sess.run(result))

print(a.graph is tf.get_default_graph())


Tensor("add_6:0", shape=(2,), dtype=float32)
[3. 5.]


## TensorFlow 计算模型

### 计算图的概念

TensorFlow 的名字中己经说明了它最重要的两个概念
Tensor: 就是张量。
Flow:  流，张量之间通过计算相互转化的过程。

所有 Te nsorFlow 的程序都可以通过类似计算图的形式来表示。
- 数据集
- 操作节点：加，减，乘，除....

### 计算图的的使用

在 TensorFlow 程序中，系统会自动维护一个默认的计算图，通过`tf.get_default_graph()`函数可以获取当前默认的计算图。
 TensorFlow 支持通过 `tf.Graph` 函数来生成新的计算图
 - TensorFlow 中的计算图不仅仅可以用来隔离张量和计算
 - 提供了管理张量和计算的机制
 - 指定运行计算的设备 `tf. Graph.device`






In [64]:
import tensorflow as tf

 #tf.Graph 函数来生成新的计算图。
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v",shape=[3],initializer=tf.zeros_initializer())
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v",shape = [3],initializer = tf.ones_initializer())
    
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse = True):
        print(sess.run(tf.get_variable("v")))
        

with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse = True):
        print(sess.run(tf.get_variable("v")))
        
        
g = tf.Graph()

# 指定运行设备
with g.device('/gpu:0'):
    result = a +b
    print(result)

[0. 0. 0.]
[1. 1. 1.]
Tensor("add_8:0", shape=(2,), dtype=float32)


##  TensorFlow 数据模型一一张量

### 张量的概念

**张量的理解**
- 所有的数据都通过张量的形式来表示
- 张量可以被简单理解为多维数组。
- 在张量中并没有真正保存数字，它保存的是如何得到这些数字的计算过程.
- TensorFlow计算的结果不是一个具体的数字，而且一个张量的结构

**张量的构成**
- TensorFlow 计算的结果不是一个具体的数字 ，而且一个张量的结构
- Tensor("add_6:0", shape=(2,), dtype=float32)
- 名字（ name ）、维度（ shape ）和类型（ type ）
- 名字（ name ）
    -  “ node:src_output ”的形式来给出；
    -  node 为节点的名称， src一output 表示当前张量来自节点的第几个输出。
    - “ add ： 。 ”就说明了 result 这个张量是计算节点“ add ” 输出的第一个结果（编号从 O 开始）。
- 维度（ shape ）
    - 这个属性描述了 一个张量的维度信息
    - shape=(2 ， ） 说明了张量 result 是一个一维数组， 这个数组的长度为 2 
- 类型（ type ）
    - 每一个张量会有一个唯一的类型。
    -  TensorFlow 会对参与运算的所有张量进行类型的检查 ， 当发现类型不匹配时会报错。 
    -  TensorFlow 支持 14 种不同的类型：实数（ tf.丑oat32 、 tf.float64 ）、整数（ tf.int8 、 tf.intl 6 、 tf.int32 、 tf.int64 、 tf.uint8 ）、布尔型 C tf.bool) 和复数（ tf.complex64 、tf.complex128 ） 
    
    
### 张量的使用

两大类型：
- 第一类用途是对中间计算结果的引用，
    - 张量可以大大提高代码的可读性
    - 通过张量来存储中间结果可以方便获取中间结果
- 当计算图构造完成之后，张量可以用来获得计算结果
    - 以使用`tf.Session().run(result)` 语句得到计算结果 


## TensorFlow 运行模型一一会话

TensorFlow 中的会话（ session ）来执行定义好的运算。会话拥有并管理 TensorFlow 程序运
行时的所有资源。

第一种模式需要明确调用会话生成函数和关闭会话函数，这种模式的代码流程如下。
- 当程序因为异常而退出时，关闭会话的函数可能就不会被执行从而导致资源泄漏 

``` (python)
sess = tf.Session()
sess.run(....)
sess.close()
```

第二种模式： Python 的上下文管理器来使用会话
- 只要将所有的计算放在 “ with ”的内部就可以。当上下文管理器退出时候会自动释放所有资源。\
- 这样既解决了因为异常退出时资源释放的问题，同时也解决了忘记调用 Sess ion.close 函数而产生的资源泄漏。

```
with tf.Session() as sess:
    sess.run(...)
    
```

交互式环境下(比如 Python 脚本或者 Jupyter 的编辑器下)，通过设置默认会话的方式来获取张量的取值更加方便。这个函数就是 `tf.lnteractiveSession` 。使用这个函数会自动将生成的会话注册为默认会话

** ConfigProto 配置**
第一个是 allow_so位_placement ，这是一个布尔型的参
数，当它为 True 时， 在以下任意一个条件成立时， GPU 上的运算可 以放到 CPU 上进行 ：

l. 运算无法在 GPU 上执行。
2. 没有 GPU 资源（比如运算被指定在第二个 GPU 上运行 ，但是机器只有一个 GPU ） 。
3. 运算输入包含对 CPU 计算结果的引用 

``` 
config  =  tf.ConfigProto(allow  soft  placement=True, 
                    log_device_placement=True) 
sessl  =  tf.InteractiveSession(config=config) 
sess2  =  tf.Session(config=config) 
```
